<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment03/seq2seq_without_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!unzip '/content/drive/MyDrive/lexicons.zip'

Archive:  /content/drive/MyDrive/lexicons.zip
   creating: lexicons/
  inflating: lexicons/ta.translit.sampled.test.tsv  
  inflating: lexicons/ta.translit.sampled.dev.tsv  
  inflating: lexicons/ta.translit.sampled.train.tsv  


In [2]:
!pip install wandb

     |████████████████████████████████| 1.8MB 7.5MB/s 
     |████████████████████████████████| 174kB 38.7MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 133kB 35.7MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=0b49f50e5d4b6c6b10dd62d286349b296959a563ad78302d3047cf742c9f0410
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=1c8df9ebd03ecc193f72fe0f119500bf624913ae59348b294e80b36d68057b8f
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [4]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from wandb.keras import WandbCallback

In [11]:
train_df = pd.read_csv("/content/lexicons/ta.translit.sampled.train.tsv", sep = '\t', header = None)
train_df = train_df.dropna(axis=0)

val_df = pd.read_csv("/content/lexicons/ta.translit.sampled.dev.tsv", sep = '\t', header = None)
val_df = val_df.dropna(axis=0)

test_df = pd.read_csv("/content/lexicons/ta.translit.sampled.test.tsv", sep = '\t', header = None)
test_df = test_df.dropna(axis=0)

In [ ]:
train_df

,0,1,2
0,ஃபியட்,fiat,2
1,ஃபியட்,phiyat,1
2,ஃபியட்,piyat,1
3,ஃபிரான்ஸ்,firaans,1
4,ஃபிரான்ஸ்,france,2
...,...,...,...
68213,ஹோல்ட்,holtt,1
68214,ஹோல்ட்,hoold,1
68215,ஹோல்ட்,hoolt,1
68216,ஹோல்ட்,hooltt,1


In [ ]:
val_df

,0,1,2
0,ஃபயர்,fire,3
1,ஃபயர்,phayar,1
2,ஃபார்,baar,1
3,ஃபார்,bar,1
4,ஃபார்,far,1
...,...,...,...
6822,ஹோலி,hoolley,1
6823,ஹோலி,hoolli,2
6824,ஹோலி,hoolly,1
6825,ஹோலி,hooly,2


In [ ]:
test_df[0]

0       ஃபார்ம்
1       ஃபார்ம்
2       ஃபார்ம்
3       ஃபார்ம்
4         ஃபேஸ்
         ...   
6859     ஹைட்ரோ
6860     ஹைட்ரோ
6861     ஹைட்ரோ
6862      ஹைதர்
6863      ஹைதர்
Name: 0, Length: 6864, dtype: object

In [12]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for input_text,target_text in zip(train_df[1][0:],train_df[0][0:]):
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
pred_target_token_index = dict([(i, char) for i, char in enumerate(target_characters)])

if ' ' not in input_token_index:
  input_token_index[' '] = len(input_token_index)

if ' ' not in target_token_index:
  target_token_index[' '] = len(target_token_index)

pred_target_token_index[len(pred_target_token_index)] = ' '

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) + 1
num_decoder_tokens = len(target_characters) + 1
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 68215
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28


In [ ]:
target_token_index

In [13]:
# Vectorize the data.
val_input_texts = []
val_target_texts = []

for val_input_text,val_target_text in zip(val_df[1][0:],val_df[0][0:]):
    val_target_text = "\t" + val_target_text + "\n"
    val_input_texts.append(val_input_text)
    val_target_texts.append(val_target_text)

val_encoder_input_data = np.zeros(
    (len(val_input_texts), max_encoder_seq_length), dtype="float32"
)
val_decoder_input_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    val_encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1 :] = target_token_index[" "]
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [14]:
# Vectorize the data.
test_input_texts = []
test_target_texts = []

for test_input_text,test_target_text in zip(test_df[1][0:],test_df[0][0:]):
    test_target_text = "\t" + test_target_text + "\n"
    test_input_texts.append(test_input_text)
    test_target_texts.append(test_target_text)

test_encoder_input_data = np.zeros(
    (len(test_input_texts), max_encoder_seq_length), dtype="float32"
)
test_decoder_input_data = np.zeros(
    (len(test_input_texts), max_decoder_seq_length), dtype="float32"
)
test_decoder_target_data = np.zeros(
    (len(test_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t] = input_token_index[char]
    test_encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    test_decoder_input_data[i, t + 1 :] = target_token_index[" "]
    test_decoder_target_data[i, t:, target_token_index[" "]] = 1.0



In [17]:
def model(cell_name, embedding_size, hidden_size, num_enc_layers, num_dec_layers, dropout, r_dropout, batch_size, num_epochs, optimizer):
  
  cell_type = { "RNN": keras.layers.SimpleRNN,
                "GRU": keras.layers.GRU,
                "LSTM": keras.layers.LSTM
  }

  learning_rate = 0.01
  decay_rate = 0

  optimizers = {"adam": tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate),
                "nadam": tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate),
                "rmsprop": tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay_rate),
                "adagrad": tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
  }

  # Define an input sequence and process it.
  encoder_inputs = keras.layers.Input(shape=(None,), name="input_1")
  embedding = keras.layers.Embedding(num_encoder_tokens, embedding_size, name="embedding_1")(encoder_inputs)
  encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_1") (embedding)
  for i in range(1, num_enc_layers):
    encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_"+str(i+1), dropout=dropout,
                                                       recurrent_dropout=r_dropout) (encoder_seq)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None,), name="input_2")
  decoder_embedding = keras.layers.Embedding(num_decoder_tokens, embedding_size, name="embedding_2")(decoder_inputs)

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_seq, *_ = cell_type[cell_name](hidden_size,return_sequences=True, return_state=True, name="decoder_1")(decoder_embedding,initial_state=encoder_state)
  for i in range(1, num_dec_layers):
    decoder_seq, *_ = cell_type[cell_name](hidden_size, return_sequences=True, return_state=True, name="decoder_"+str(i+1),dropout=dropout,
                                      recurrent_dropout=r_dropout) (decoder_seq)

  decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation="softmax", name="dense_1")(decoder_seq)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(
    optimizer=optimizers[optimizer], loss="categorical_crossentropy", metrics=["accuracy"]
  )
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=num_epochs,
      validation_data=([val_encoder_input_data, val_decoder_input_data],
                      val_decoder_target_data),
      callbacks=[WandbCallback()]
  )

In [20]:
def swp():

  hyperparameter_defaults = dict(
      cell_name = "LSTM",
      embedding_size = 256,
      hidden_size = 32,
      num_enc_layers = 2,
      num_dec_layers = 2,
      dropout = 0.2,
      r_dropout = 0.2,
      batch_size = 512,
      num_epochs = 10,
      optimizer = "adam"
  )

  wandb.init(project="Assignment 3 without Attention", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "{}_cell_{}_embSize_{}_hiddenSize_{}_encLayers_{}_decLayers_{}_dropout_{}_rDropout_{}_batchSize_{}_epochs_{}_opt".format(config.cell_name, 
                    config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, config.num_epochs, 
                    config.optimizer)
  
  model(config.cell_name, config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, 
         config.num_epochs, config.optimizer)
  wandb.run.finish()

In [ ]:
sweep_config = {
  "name": "My Sweep",
  "method": "random",
  "project": "Assignment02",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "cell_name": {
            "values": ['RNN', 'LSTM', 'GRU']
        },
        "embedding_size": {
            "values":[16, 32, 64, 128, 256]
        }, 
        "hidden_size": {
            "values":[16, 32, 64, 128, 256]
        },
        "num_enc_layers":{
            "values":[1, 2, 3]
        },
        "num_dec_layers":{
            "values":[1, 2, 3]
        },  
        "dropout":{
            "values":[0, 0.2, 0.3, 0.4]
        },
        "r_dropout": {
            "values":[0, 0.2, 0.3, 0.4]
        },
        "batch_size": {
            "values":[32,64,128,256]
        },
        "num_epochs": {
            "values":[5, 10, 15]
        },
        "optimizer": {
            "values":['adam', 'nadam','rmsprop','adagrad']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 63fhsmfp
Sweep URL: https://wandb.ai/kodikarthik21/uncategorized/sweeps/63fhsmfp


In [ ]:
#wandb.agent(sweep_id, swp)

In [ ]:
#-----------------LOADING THE BEST MODEL FROM THE SWEEP-------------------------
model = tf.keras.models.load_model(wandb.restore('model-best.h5', run_path="kodikarthik21/Assignment-3/wze9pcb0").name)
model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [21]:
#----------------------USE FOR SAMPLE-------------------------------------------
model = tf.keras.models.load_model(wandb.restore('model-best.h5', run_path="kodikarthik21/Assignment 3 without Attention/1513pa90").name)
test_results = model.predict([test_encoder_input_data, test_decoder_input_data])

In [39]:
test_pred = {}
for i in range(test_results.shape[0]):
  for j in range(26):
    if(j==0):
      test_pred[i] = pred_target_token_index[np.argmax(test_results[i],axis = 1)[j]]
    else:
      test_pred[i] = test_pred[i] + pred_target_token_index[np.argmax(test_results[i],axis = 1)[j]]
    #print(pred_target_token_index[np.argmax(test_results[i],axis = 1)[j]])
  test_pred[i] = test_pred[i].strip('\n')

In [38]:
test_print= {}
count =0;
for i in range(6864):
  test_print[i] = [test_df[0][i],test_df[1][i],test_df[2][i],test_pred[i]]
  if(test_df[0][i]==test_pred[i]):
    count = count+1
print(test_print)
print(count)

{0: ['ஃபார்ம்', 'faarm', 1, 'பபிர்ட்'], 1: ['ஃபார்ம்', 'farm', 2, 'பபிர்ட்'], 2: ['ஃபார்ம்', 'form', 1, 'பபிர்ட்'], 3: ['ஃபார்ம்', 'hpaarm', 1, 'அபார்ட்'], 4: ['ஃபேஸ்', 'face', 3, 'பபாச்'], 5: ['ஃபேஸ்', 'hpaes', 1, 'அபா\n்'], 6: ['ஃபேஸ்', 'pace', 2, 'பபா\n்'], 7: ['ஃபேஸ்', 'paes', 1, 'பபா\n்'], 8: ['ஃபேஸ்', 'phase', 1, 'பபா\n்'], 9: ['அஇஅதிமுக', 'aeathimuka', 1, 'அததடிரு\n்'], 10: ['அஇஅதிமுக', 'aiathimuka', 1, 'அததடிரு\n்'], 11: ['அஇஅதிமுக', 'ayiathimuka', 1, 'அததடிரிக்'], 12: ['அகத்தி', 'agaththi', 3, 'அத்்தி'], 13: ['அகத்தி', 'akaththi', 2, 'அத்்தி'], 14: ['அகத்திக்கீரை', 'agaththikkeerai', 3, 'அதி்திரளகாட்'], 15: ['அகத்திக்கீரை', 'akaththikkeerai', 2, 'அதி்திரளகாட்'], 16: ['அகமதாபாத்', 'agamadhabaath', 1, 'அத்ாாத்ர்'], 17: ['அகமதாபாத்', 'agamathaabaath', 1, 'அத்ாாத்ர்'], 18: ['அகமதாபாத்', 'ahamadabad', 1, 'அதராாத்\n்'], 19: ['அகமதாபாத்', 'ahemadaabad', 1, 'அதராாத்\n்'], 20: ['அகமதாபாத்', 'ahmadabad', 1, 'அதராாத்\n்'], 21: ['அகமதாபாத்', 'ahmedabad', 3, 'அதராாத்\n்'], 22: ['அகமதாபாத்'

In [ ]:
test_pred

In [11]:
def model_without_wandb(cell_name, embedding_size, hidden_size, num_enc_layers, num_dec_layers, dropout, batch_size, num_epochs, optimizer, r_dropout = 0.4):
  
  cell_type = { "RNN": keras.layers.SimpleRNN,
                "GRU": keras.layers.GRU,
                "LSTM": keras.layers.LSTM
  }

  learning_rate = 0.01
  decay_rate = 0

  optimizers = {"adam": tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate),
                "nadam": tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate),
                "rmsprop": tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay_rate),
                "adagrad": tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
  }

  # Define an input sequence and process it.
  encoder_inputs = keras.layers.Input(shape=(None,), name="input_1")
  embedding = keras.layers.Embedding(num_encoder_tokens, embedding_size, name="embedding_1")(encoder_inputs)
  encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_1") (embedding)
  for i in range(1, num_enc_layers):
    encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_"+str(i+1), dropout=dropout, 
                                                       recurrent_dropout = r_dropout) (encoder_seq)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None,), name="input_2")
  decoder_embedding = keras.layers.Embedding(num_decoder_tokens, embedding_size, name="embedding_2")(decoder_inputs)

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_seq, *_ = cell_type[cell_name](hidden_size,return_sequences=True, return_state=True, name="decoder_1")(decoder_embedding,initial_state=encoder_state)
  for i in range(1, num_dec_layers):
    decoder_seq, *_ = cell_type[cell_name](hidden_size, return_sequences=True, return_state=True, name="decoder_"+str(i+1),dropout=dropout,
                                           recurrent_dropout=r_dropout) (decoder_seq)

  decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation="softmax", name="dense_1")(decoder_seq)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(
    optimizer=optimizers[optimizer], loss="categorical_crossentropy", metrics=["accuracy"]
  )
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=num_epochs,
      validation_data=([val_encoder_input_data, val_decoder_input_data],
                      val_decoder_target_data)
  )

  model.save("s2s")
  return model

In [ ]:
def test(model):
#-----------MODEL EVALUATION WITH TEST DATA-------------------------------------
  wandb.init(project="Assignment-3")
  wandb.run.name = "Test_run_for_best_model"
  results = model.evaluate([test_encoder_input_data, test_decoder_input_data],
                      test_decoder_target_data,callbacks=[WandbCallback()])
  wandb.log({'test_loss': results[0], 'test_accuracy': results[1]})

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
latent_dim = 128
encoder_inputs = model.input[0]  # input_1
state_h_enc, state_c_enc = model.layers[3].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[7]
print(decoder_lstm)
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[8]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
model = model_without_wandb("LSTM", 256, 64, 2, 2, 0.5, 512, 5, "rmsprop")

Epoch 1/5
134/134 [==============================] - 78s 321ms/step - loss: 1.3053 - accuracy: 0.6788 - val_loss: 0.7072 - val_accuracy: 0.7942
Epoch 2/5
134/134 [==============================] - 42s 316ms/step - loss: 0.7242 - accuracy: 0.7870 - val_loss: 0.6370 - val_accuracy: 0.8091
Epoch 3/5
134/134 [==============================] - 43s 320ms/step - loss: 0.6490 - accuracy: 0.8062 - val_loss: 0.5782 - val_accuracy: 0.8269
Epoch 4/5
134/134 [==============================] - 43s 321ms/step - loss: 0.6084 - accuracy: 0.8166 - val_loss: 0.5588 - val_accuracy: 0.8308
Epoch 5/5
134/134 [==============================] - 42s 314ms/step - loss: 0.5870 - accuracy: 0.8219 - val_loss: 0.5469 - val_accuracy: 0.8369


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [ ]:
model = keras.models.load_model("s2s")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    6912        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_1 (LSTM)                [(None, None, 64), ( 82176       embedding_1[0][0]                
______________________________________________________________________________________________

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(16,), name="input_3")
decoder_state_input_c = keras.Input(shape=(16,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence